In [12]:
%load_ext watermark
%watermark -v -m  -u -n -p pandas,topicpy,plotly -a Filippo_Valle -g -r -b -w

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: Filippo_Valle

Last updated: Fri Oct 18 2024

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.5.0

pandas : 1.5.0
topicpy: 0.3.0
plotly : 5.20.0

Compiler    : GCC 10.4.0
OS          : Linux
Release     : 6.8.0-47-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

Git hash: f560f69d221e93d83e6db5bad232b0e992836786

Git repo: git@github.com:fvalle1/topics.git

Git branch: HEAD

Watermark: 2.4.3



In [13]:
from topicpy.hsbmpy import get_scores, normalise_score, get_scores_shuffled
import pandas as pd
import plotly.graph_objects as go
import os
import numpy as np

In [14]:
labels = ["SMTS"]
alorithms = ["topsbm", "lda", "tm", "wgcna", "hierarchical-log"]

In [47]:
data = {}
for stat in range(0,10):
    name=f"random{stat}{stat}"
    working_dir = name
    df_files = pd.read_csv(f"{working_dir}/files.dat", index_col=[0], header=[0]).dropna(axis=1, how='all').dropna(axis=0, how='all')
    samples = pd.read_csv(f"{working_dir}/topsbm/topsbm_level_0_clusters.csv", header=[0]).astype(str).values.ravel()
    samples=samples[samples!="nan"]
    df_files = df_files.reindex(index=samples).dropna(how="all", axis=0).fillna("unknown")

    scores = {}
    for alg in alorithms:
        if alg not in data.keys():
            data[alg]=[]
        scores[alg] = get_scores(working_dir, labels, df_files=df_files, algorithm = alg, verbose=False)[labels[0]]
    # scores['shuffle'] = get_scores_shuffled(working_dir, df_files, label=labels[0], algorithm="topsbm")
    # scores['shuffle']["V"]=np.average([get_scores_shuffled(working_dir, df_files, label=labels[0], algorithm="topsbm")["V"] for _ in range(5)], axis=0)
    # norm_scores = scores.copy()
    # normalise_score(norm_scores, base_algorithm="shuffle")
    for alg in alorithms:
        data[alg].append(scores[alg])

In [48]:
data_hk = {}
df_files = pd.read_csv(f"../gtexhk/files.dat", index_col=[0], header=[0]).dropna(axis=1, how='all').dropna(axis=0, how='all')
samples = pd.read_csv(f"../gtexhk/topsbm/topsbm_level_0_clusters.csv", header=[0]).astype(str).values.ravel()
samples=samples[samples!="nan"]
# df_files = df_files.reindex(index=samples).dropna(how="all", axis=0).fillna("unknown")
scores = {}
for alg in alorithms:
    if alg not in data_hk.keys():
        data_hk[alg]=[]
    scores[alg] = get_scores(f"../gtexhk", labels, df_files=df_files, algorithm = alg, verbose=False)[labels[0]]
# scores['shuffle'] = get_scores_shuffled(f"../gtexhk", df_files, label=labels[0], algorithm="topsbm")
# scores['shuffle']["V"] = np.average([get_scores_shuffled(f"../gtexhk", df_files, label=labels[0], algorithm="topsbm")["V"] for _ in range(5)], axis=0)
# norm_scores = scores.copy()
# normalise_score(norm_scores, base_algorithm="shuffle")
for alg in alorithms:
    data_hk[alg].append(scores[alg])

In [49]:
data_hv = {}
df_files = pd.read_csv(f"../gtex10/files.dat", index_col=[0], header=[0]).dropna(axis=1, how='all').dropna(axis=0, how='all')
samples = pd.read_csv(f"../gtex10/topsbm/topsbm_level_0_clusters.csv", header=[0]).astype(str).values.ravel()
samples=samples[samples!="nan"]
# df_files = df_files.reindex(index=samples).dropna(how="all", axis=0).fillna("unknown")
scores = {}
for alg in alorithms:
    if alg not in data_hv.keys():
        data_hv[alg]=[]
    scores[alg] = get_scores(f"../gtex10", labels, df_files=df_files, algorithm = alg, verbose=False)[labels[0]]
# scores['shuffle'] = get_scores_shuffled(f"../gtex10", df_files, label=labels[0], algorithm="topsbm")
# scores['shuffle']["V"] = np.average([get_scores_shuffled(f"../gtex10", df_files, label=labels[0], algorithm="topsbm")["V"] for _ in range(5)], axis=0)
# norm_scores = scores.copy()
# normalise_score(norm_scores, base_algorithm="shuffle")
for alg in alorithms:
    data_hv[alg].append(scores[alg])

In [67]:
fig = go.Figure()

for alg in alorithms:
    fig.add_trace(go.Box(y = list(map(lambda x: x["V"][np.argmin(np.abs(np.array(x["xl"])-10))], data[alg])), name=alg))
    fig.add_trace(go.Box(y = list(map(lambda x: x["V"][np.argmin(np.abs(np.array(x["xl"])-10))], data_hk[alg])), line_width=5, line_color="red", name=alg))
    fig.add_trace(go.Box(y = list(map(lambda x: x["V"][np.argmin(np.abs(np.array(x["xl"])-10))], data_hv[alg])), line_width=5, line_color="blue", name=alg))

adict = {"tickfont_size":40, "titlefont_size":40}
fig.update_layout({"yaxis":{"title": "NMI score", "range":[0., 1], **adict}, "xaxis":adict, "width":1224, "height": 1024})
fig.show()
fig.write_image("random_scores.pdf")